<a href="https://colab.research.google.com/github/luigiantonelli/DeepLearning-Project/blob/main/Deep_Learning_Project_Antonelli_Cuconasu_Gaudenzi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installations and imports

In [1]:
!pip install pytorch-lightning --quiet
!pip install torchmetrics --quiet
!pip install gdown==4.5.4 --no-cache-dir --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 826.4/826.4 KB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.2/517.2 KB 25.3 MB/s eta 0:00:00


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader, random_split
import torchmetrics
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle as pkl
from tqdm.notebook import tqdm
import pytorch_lightning as pl 
import math
from math import sqrt
import pickle
from typing import *
import gdown

# Dataset

In [4]:
"""
url = "https://drive.google.com/drive/folders/1-6MRkFoSSRJqeKgcMXm3PeA159KzHuB_?usp=sharing"
gdown.download_folder(url = url, quiet = True, use_cookies = False, remaining_ok=True)
"""

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [5]:
#dataset_folder_path = "/content/drive/MyDrive/Colab Notebooks/Deep Learning/DeepLearningProject-Shared"
dataset_folder_path = "/content/drive/MyDrive/Deep_Learning_Project"
os.chdir(dataset_folder_path)

In [6]:
!ls

algebra__linear_1d  datasets  mathematics_dataset-v1.0	modules.txt


In [ ]:
def read_dataset(text_path: str, lowercase: bool=True) -> Tuple[List[str], List[str]]:
    questions = []
    answers = []

    with open(text_path) as f:
        for idx, line in enumerate(f):
            if lowercase:
                if idx % 2 == 0: # Questions
                    questions.append(line.rstrip().lower()) 
                else:
                    answers.append(line.rstrip().lower())

    return questions, answers

In [ ]:
def get_vocabulary(lists_of_texts: List[List[str]]):
    unified_text = []
    
    for l in lists_of_texts:
        unified_text += l

    return Counter(" ".join(unified_text)).keys()

In [ ]:
import glob
# Get all files
folders = ['extrapolate', 'interpolate', 'train-easy', 'train-medium', 'train-hard']
files = []

for fold in folders:
    files += glob.glob(f"./mathematics_dataset-v1.0/{fold}/*.txt")

In [ ]:
files[:5]

['./mathematics_dataset-v1.0/extrapolate/arithmetic__add_sub_multiple_longer.txt',
 './mathematics_dataset-v1.0/extrapolate/algebra__polynomial_roots_big.txt',
 './mathematics_dataset-v1.0/extrapolate/arithmetic__add_or_sub_big.txt',
 './mathematics_dataset-v1.0/extrapolate/arithmetic__div_big.txt',
 './mathematics_dataset-v1.0/extrapolate/arithmetic__mul_div_multiple_longer.txt']

In [ ]:
vocabulary = {}
all_lists = []

In [ ]:
i = 0
for f in files:
    train, test = read_dataset(f)
    all_lists += train
    all_lists += test

    vocabulary = vocabulary | get_vocabulary(all_lists)
    all_lists = []

    if i % 10 == 0:
        with open('./datasets/vocabulary.pkl', 'wb') as f:
            pickle.dump(vocabulary, f)

In [ ]:
vocabulary

In [7]:
with open('./datasets/vocabulary.pkl', 'rb') as f:
    vo = pickle.load(f)

In [8]:
vocabulary = vo

In [9]:
len(vocabulary)

54

In [ ]:
len(all_lists)

20000

In [ ]:
!ls ./mathematics_dataset-v1.0/train-easy

algebra__linear_1d_composed.txt
algebra__linear_1d.txt
algebra__linear_2d_composed.txt
algebra__linear_2d.txt
algebra__polynomial_roots_composed.txt
algebra__polynomial_roots.txt
algebra__sequence_next_term.txt
algebra__sequence_nth_term.txt
arithmetic__add_or_sub_in_base.txt
arithmetic__add_or_sub.txt
arithmetic__add_sub_multiple.txt
arithmetic__div.txt
arithmetic__mixed.txt
arithmetic__mul_div_multiple.txt
arithmetic__mul.txt
arithmetic__nearest_integer_root.txt
arithmetic__simplify_surd.txt
calculus__differentiate_composed.txt
calculus__differentiate.txt
comparison__closest_composed.txt
comparison__closest.txt
comparison__kth_biggest_composed.txt
comparison__kth_biggest.txt
comparison__pair_composed.txt
comparison__pair.txt
comparison__sort_composed.txt
comparison__sort.txt
measurement__conversion.txt
measurement__time.txt
numbers__base_conversion.txt
numbers__div_remainder_composed.txt
numbers__div_remainder.txt
numbers__gcd_composed.txt
numbers__gcd.txt
numbers__is_factor_composed

In [ ]:
algebra_path = "./mathematics_dataset-v1.0/train-easy/algebra__linear_1d.txt"
probability_path = "./mathematics_dataset-v1.0/train-easy/probability__swr_p_level_set.txt"
prime_path = "./mathematics_dataset-v1.0/train-easy/numbers__is_prime.txt"

In [ ]:
questions_easy_algebra, answers_easy_algebra = read_dataset(algebra_path)
questions_easy_probability, answers_easy_probability = read_dataset(probability_path)
questions_easy_prime, answers_easy_prime = read_dataset(prime_path)

In [ ]:
from collections import Counter
lis = " ".join(questions_easy_prime)
Counter(lis)

In [ ]:
t = []
a = questions_easy_algebra + answers_easy_algebra

In [ ]:
a[-4]

'-7'

In [ ]:
a = get_vocabulary([questions_easy_algebra, answers_easy_algebra])

In [ ]:
a

dict_keys(['s', 'o', 'l', 'v', 'e', ' ', '0', '=', '4', '*', 'b', '+', '1', '5', 'f', 'r', '.', '-', '3', 'd', 'h', '9', '2', 'm', '8', '7', 'a', '6', 'w', 'c', 'z', 'j', 'n', 'y', 'x', 'u', 'g', 'k', 'q', 'i', 'p', 't'])

In [ ]:
import string
a = string.printable

In [ ]:
string.ascii_uppercase

'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

In [ ]:
len(string.ascii_lowercase) + 10 + ['%', '&', '(', ')', ]

26

In [ ]:
a

'0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ \t\n\r\x0b\x0c'

'0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
len(a[:-6]) - len(string.ascii_uppercase)

68

In [ ]:
"""
#insert <bos>, <eos> and <pad>
it = 0
questions, answers = [], []
seq_len = 256
answer_len = 50
for qa in train:
    q = qa['question']
    a = qa['answer']
    question = ['<bos>']
    answer = ['<bos>']
    for i in range(1, len(q) - 1):
        question.append(q[i])
    while i < seq_len - 1:
        question.append('<pad>')
        i += 1
    question.append('<eos>')
    questions.append(question)
    for j in range(1, len(a) - 1):
        answer.append(a[j])
    while j < answer_len - 1:
        answer.append('<pad>')
        j += 1
    answer.append('<eos>')
    answers.append(answer)
    it += 1
    if it % 10000 == 0:
        print(f"iteration {it}")
"""

In [ ]:
list(vocabulary)

['5',
 'u',
 '3',
 ',',
 'l',
 '-',
 '9',
 'i',
 'm',
 '.',
 "'",
 'r',
 '?',
 'o',
 'z',
 ' ',
 '8',
 '0',
 'a',
 'g',
 'w',
 '+',
 '>',
 'p',
 'e',
 '*',
 'f',
 '4',
 's',
 'k',
 '/',
 '7',
 'v',
 't',
 'b',
 '(',
 'x',
 '=',
 '6',
 ':',
 '<',
 '}',
 'n',
 'y',
 '!',
 'c',
 '2',
 ')',
 'j',
 '1',
 'h',
 'd',
 'q',
 '{']

In [10]:
def create_vocabulary_from_set(voc: set):
    vocabulary = {'<bos>': 0, '<eos>': 1, '<unk>': 2, '<pad>': 3}
    i = 4
    for v in voc:
        vocabulary[v] = i
        i += 1
    return vocabulary

In [11]:
v = create_vocabulary_from_set(vocabulary)

In [ ]:
def read_dataset(text_path: str, lowercase: bool=True) -> Tuple[List[str], List[str]]:
    questions = []
    answers = []

    with open(text_path) as f:
        for idx, line in enumerate(f):
            if lowercase:
                if idx % 2 == 0: # Questions
                    questions.append(line.rstrip().lower()) 
                else:
                    answers.append(line.rstrip().lower())

    return questions, answers

In [142]:
class Mathematics_Dataset(Dataset):
    def __init__(self, modules: List[str], vocabulary: dict):
        self.modules = modules
        self.questions = []
        self.answers = []
        for m in self.modules:
            q_m, a_m = self.read_dataset(m)
            self.questions += q_m
            self.answers += a_m
        self.max_len_question = 160 #forse serve multiplo di num_heads
        self.max_len_answer = 30
        self.vocabulary = vocabulary

    def read_dataset(self, text_path: str, lowercase: bool=True) -> Tuple[List[str], List[str]]:
        questions = []
        answers = []
        with open(text_path, 'r') as f:
            for idx, line in enumerate(f):
                if lowercase:
                    if idx % 2 == 0: # Questions
                        questions.append(line.rstrip().lower()) 
                    else: #Answers
                        answers.append(line.rstrip().lower())
        return questions, answers

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        assert idx < len(self.questions)
        q, a = self.questions[idx], self.answers[idx]
        question, answer = np.zeros(self.max_len_question + 2), np.zeros(self.max_len_question + 2) 
        question[0] = self.vocabulary['<bos>']
        answer[0] = self.vocabulary['<bos>']

        for i in range(len(q)):
            c = '<unk>'
            if q[i] in self.vocabulary:
                c = q[i]
            question[i + 1] = self.vocabulary[c]
        i += 1
        while i < self.max_len_question:
            question[i + 1] = self.vocabulary['<pad>']
            i += 1
        question[self.max_len_question + 1] = self.vocabulary['<eos>']
    
        for j in range(len(a)): #bos caratteri pad eos pad MODIFICA POSSIBILE
            c = '<unk>'
            if a[j] in self.vocabulary:
                c = a[j]
            answer[j + 1] = self.vocabulary[c]
        j += 1
        while j < self.max_len_answer: 
            answer[j + 1] = self.vocabulary['<pad>']
            j += 1
        answer[self.max_len_answer + 1] = self.vocabulary['<eos>']
        for j in range(self.max_len_answer + 2, self.max_len_question + 2):
            answer[j] = self.vocabulary['<pad>']
        question = torch.from_numpy(question).long()
        answer = torch.from_numpy(answer).long()
        return question, answer

In [ ]:
import os
cwd = os.getcwd()
cwd

'/content/drive/MyDrive/Deep_Learning_Project'

In [ ]:
!ls

algebra__linear_1d  datasets  mathematics_dataset-v1.0	modules.txt


In [143]:
d = Mathematics_Dataset(['./mathematics_dataset-v1.0/train-easy/algebra__linear_1d.txt'], v)

In [151]:
q, a = d.__getitem__(8)

In [150]:
q, a

(tensor([ 0, 48, 55, 32, 39, 13, 12, 42, 18, 40, 47, 36, 12, 34, 12, 42, 40, 47,
         36, 12, 42, 12, 18, 46, 12,  6, 55, 10, 12, 36, 56,  3,  3,  3,  3,  3,
          3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
          3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
          3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
          3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
          3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
          3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
          3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  1]),
 tensor([ 0, 18,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
          3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  1,  3,  3,  3,  3,
          3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
          3,  3,  3,  3,  

In [ ]:
q.shape

torch.Size([162])

In [ ]:
!ls

algebra__linear_1d  datasets  mathematics_dataset-v1.0	modules.txt


In [153]:
class Mathematics_DataModule(pl.LightningDataModule):
    def __init__(self, modules: List[str], batch_size: int = 32):
        super().__init__()
        self.modules = modules
        self.batch_size = batch_size
        self.load_vocabulary()
    
    def load_vocabulary(self):
        with open('./datasets/vocabulary.pkl', 'rb') as f:
            v = pickle.load(f)
        self.vocabulary = create_vocabulary_from_set(v)

    def setup(self):
        self.math = Mathematics_Dataset(self.modules, self.vocabulary)
        self.math_train, self.math_val, self.math_test = random_split(self.math, [0.75, 0.05, 0.20])
    
    def train_dataloader(self):
        return DataLoader(self.math_train, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):                                                              
        return DataLoader(self.math_val, batch_size=self.batch_size, shuffle=True)

    def test_dataloader(self):
        return DataLoader(self.math_test, batch_size=self.batch_size, shuffle=True)

    def teardown(self, stage: str):
        # Used to clean-up when the run is finished
        pass

In [ ]:
dm = Mathematics_DataModule(['./mathematics_dataset-v1.0/train-easy/algebra__linear_1d.txt'], batch_size = 64)

In [ ]:
dm.setup()

In [ ]:
dm.train_dataloader()

In [ ]:
i = 0
for s in dm.train_dataloader():
    print(s)
    i += 1
    if i == 2:
        break

[tensor([[ 0., 43., 21.,  ...,  3.,  3.,  1.],
        [ 0., 43., 21.,  ...,  3.,  3.,  1.],
        [ 0., 43., 21.,  ...,  3.,  3.,  1.],
        ...,
        [ 0., 43., 21.,  ...,  3.,  3.,  1.],
        [ 0., 43., 21.,  ...,  3.,  3.,  1.],
        [ 0., 43., 21.,  ...,  3.,  3.,  1.]], dtype=torch.float64), tensor([[ 0., 12., 37.,  ...,  3.,  3.,  1.],
        [ 0., 12., 40.,  ...,  3.,  3.,  1.],
        [ 0., 12., 51.,  ...,  3.,  3.,  1.],
        ...,
        [ 0., 18.,  3.,  ...,  3.,  3.,  1.],
        [ 0., 12., 51.,  ...,  3.,  3.,  1.],
        [ 0., 12., 36.,  ...,  3.,  3.,  1.]], dtype=torch.float64)]
[tensor([[ 0., 43., 21.,  ...,  3.,  3.,  1.],
        [ 0., 43., 21.,  ...,  3.,  3.,  1.],
        [ 0., 43., 21.,  ...,  3.,  3.,  1.],
        ...,
        [ 0., 43., 21.,  ...,  3.,  3.,  1.],
        [ 0., 43., 21.,  ...,  3.,  3.,  1.],
        [ 0., 43., 21.,  ...,  3.,  3.,  1.]], dtype=torch.float64), tensor([[ 0., 40.,  3.,  ...,  3.,  3.,  1.],
        [ 0., 33

# Modules

In [ ]:
#forse meglio definire una stable softmax
"""
la x emb_d @ emb_d x lq
la x lq (forse errore perché la maschera è la x la)
"""

In [119]:
def dot_product_attention(query, key, value, sqrt_q, mask):
    t = torch.matmul(query, key.transpose(-2, -1))/sqrt_q
    print(f"query shape: {query.shape}")
    print(f"key.T shape: {key.transpose(-2, -1).shape}")
    print(f"t shape: {t.shape}")
    print(f"mask shape: {mask.shape}")
    if mask is not None:
      t = t.masked_fill_(mask == False, -1e-10) #-1e-10 acts like -infinity, so that the softmax will consider these tokens less important
    return torch.matmul(F.softmax(t, dim = -1), value)

In [120]:
class MultiHeadAttention(nn.Module): 
    def __init__(self, embedding_dim, num_heads, batch_size, tp_attention = False):
        super(MultiHeadAttention, self).__init__()
        assert embedding_dim % num_heads == 0
        self.tp_attention = tp_attention
        self.dim_head = embedding_dim // num_heads #single head dimension
        self.sqrt_q = sqrt(self.dim_head)
        self.num_heads = num_heads
        self.batch_size = batch_size
        self.W_q = nn.Linear(embedding_dim, embedding_dim, bias = True) #stack of num_heads matrices of dimension (d, dim_head), one for each head
        self.W_k = nn.Linear(embedding_dim, embedding_dim, bias = True)
        self.W_v = nn.Linear(embedding_dim, embedding_dim, bias = True)
        self.W_o = nn.Linear(embedding_dim, embedding_dim, bias = True)
        if self.tp_attention:
            self.W_r = nn.Linear(embedding_dim, embedding_dim, bias = True) #ruolo

    def forward(self, query, key, value, mask): #query, key, value
        q = self.W_q(query).view(self.batch_size, -1, self.num_heads, self.dim_head).transpose(1, 2)
        k = self.W_k(key).view(self.batch_size, -1, self.num_heads, self.dim_head).transpose(1, 2)
        v = self.W_v(value).view(self.batch_size, -1, self.num_heads, self.dim_head).transpose(1, 2)
        attention_value = dot_product_attention(q, k, v, self.sqrt_q, mask)
        if self.tp_attention:
            role = self.W_r(query).view(self.batch_size, -1, self.num_heads, self.dim_head).transpose(1, 2)
            attention_value *= role  #element-wise product between attention value and role before the final projection
        return self.W_o(attention_value.transpose(1, 2).contiguous().view(self.batch_size, -1, self.num_heads*self.dim_head))

In [ ]:
"""
class TP_MultiHeadAttention(nn.Module):
    def __init__(self, embedding_dim, num_heads, batch_size):
        super(TP_MultiHeadAttention, self).__init__()
        assert embedding_dim % num_heads == 0
        self.dim_head = embedding_dim // num_heads #single head dimension
        self.sqrt_q = sqrt(self.dim_head)
        self.num_heads = num_heads
        self.batch_size = batch_size
        self.W_q = nn.Linear(embedding_dim, embedding_dim, bias = True) #stack of num_heads matrices of dimension (d, dim_head), one for each head
        self.W_k = nn.Linear(embedding_dim, embedding_dim, bias = True)
        self.W_v = nn.Linear(embedding_dim, embedding_dim, bias = True)
        self.W_o = nn.Linear(embedding_dim, embedding_dim, bias = True)

    def forward(self, query, key, value, mask = None): #query, key, value
        q = self.W_q(query).view(self.batch_size, -1, self.num_heads, self.dim_head).transpose(1, 2)
        k = self.W_k(key).view(self.batch_size, -1, self.num_heads, self.dim_head).transpose(1, 2)
        v = self.W_v(value).view(self.batch_size, -1, self.num_heads, self.dim_head).transpose(1, 2)
        role = self.W_r(query).view(self.batch_size, -1, self.num_heads, self.dim_head).transpose(1, 2)
        attention_value = dot_product_attention(q, k, v, self.sqrt_q, mask).transpose(1, 2) 
        return self.W_o((attention_value * role).contiguous().view(self.batch_size, -1, self.num_heads*self.dim_head)) #element-wise product between attention value and role before the final projection
"""

In [121]:
class TransformerBlock(nn.Module):
    def __init__(self, embedding_dim, num_heads, batch_size, hidden_size = None, dropout=0.2, tp_attention = False):
        super(TransformerBlock, self).__init__()
        self.embedding_dim = embedding_dim
        self.num_heads = num_heads
        self.batch_size = batch_size
        self.attention = MultiHeadAttention(embedding_dim, num_heads, batch_size, tp_attention)
        self.norm1 = nn.LayerNorm(embedding_dim)
        self.dropout1 = nn.Dropout(dropout)
        self.norm2 = nn.LayerNorm(embedding_dim)
        self.dropout2 = nn.Dropout(dropout)
        hidden_size = 4*embedding_dim if hidden_size is None else hidden_size
        self.ff = nn.Sequential(nn.Linear(embedding_dim, hidden_size, bias = True), 
                                nn.ReLU(inplace = True),
                                nn.Linear(hidden_size, embedding_dim, bias = True))

    def forward(self, query, key, value, mask): #query, key, value
        x = query + self.attention(query, key, value, mask) #query as res conn because the decoder block requires it and it doesn't matter for encoder blocks
        x = self.dropout1(self.norm1(x))
        x = x + self.ff(x)
        x = self.dropout2(self.norm2(x))
        return x

In [122]:
class DecoderBlock(nn.Module):
    def __init__(self, embedding_dim, num_heads, batch_size, hidden_size, dropout = 0.2, tp_attention = False):
        super(DecoderBlock, self).__init__()
        self.masked_attention = MultiHeadAttention(embedding_dim, num_heads, batch_size, tp_attention)
        self.norm = nn.LayerNorm(embedding_dim)
        self.dropout = nn.Dropout(dropout)
        self.transformer_block = TransformerBlock(embedding_dim, num_heads, batch_size, hidden_size, dropout, tp_attention)

    def forward(self, output_encoder, src_mask, y, trg_mask):
        y = y + self.masked_attention(y, y, y, trg_mask) #masked attention (y = query = key = value) + residual connection
        y = self.dropout(self.norm(y))
        return self.transformer_block(y, output_encoder, output_encoder, src_mask)#query from the masked mha and key and value from the encoder

In [123]:
class PositionalEncoding(nn.Module):
    def __init__(self, embedding_dim, max_len = 5000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, embedding_dim)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, embedding_dim, 2) * -(math.log(10000.0) / embedding_dim))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        return x + Variable(self.pe[:, :x.size(1)], requires_grad = False)

In [124]:
class TransformerEncoder(nn.Module):
    def __init__(self, embedding_dim, num_heads, batch_size, hidden_size, dropout, num_blocks = 6, tp_attention = False):
        super(TransformerEncoder, self).__init__()
        self.embedding_dim = embedding_dim
        self.num_heads = num_heads
        self.batch_size = batch_size
        self.encoder = nn.ModuleList(
            [TransformerBlock(embedding_dim, num_heads, batch_size, hidden_size, dropout, tp_attention) for _ in range(num_blocks)]
            )

    def forward(self, x, mask): 
        for block in self.encoder:
            x = block(x, x, x, mask)
        return x

In [125]:
class TransformerDecoder(nn.Module):
    def __init__(self, embedding_dim, num_heads, batch_size, hidden_size, dropout = 0.2, num_blocks = 6, tp_attention = False):
        super(TransformerDecoder, self).__init__()
        self.embedding_dim = embedding_dim
        self.num_heads = num_heads
        self.batch_size = batch_size
        self.decoder = nn.ModuleList(
            [DecoderBlock(embedding_dim, num_heads, batch_size, hidden_size, dropout, tp_attention) for _ in range(num_blocks)]
            )

    def forward(self, output_encoder, src_mask, y, trg_mask): 
        for block in self.decoder:
            y = block(output_encoder, src_mask, y, trg_mask)
        return y

In [126]:
class Transformer(pl.LightningModule):
    def __init__(self, embedding_dim, num_heads, batch_size, hidden_size, dropout, vocabulary_size, pad_id = 3, num_blocks_encoder = 6, num_blocks_decoder = 6, tp_attention = False):
        super(Transformer, self).__init__()
        self.embedding_dim = embedding_dim
        self.num_heads = num_heads
        self.batch_size = batch_size
        self.pad_id = pad_id
        self.token_embedding = nn.Embedding(vocabulary_size, embedding_dim, padding_idx = self.pad_id)
        self.positional_embedding = PositionalEncoding(embedding_dim)
        self.encoder = TransformerEncoder(embedding_dim, num_heads, batch_size, hidden_size, dropout, num_blocks_encoder, tp_attention)
        self.decoder = TransformerDecoder(embedding_dim, num_heads, batch_size, hidden_size, dropout, num_blocks_decoder, tp_attention)
        self.to_logits = nn.Linear(embedding_dim, vocabulary_size)
        
        self.max_len_answer = 32
        self.accuracy = torchmetrics.Accuracy(task="multiclass", num_classes=vocabulary_size, ignore_index = self.pad_id)

    def create_trg_mask(self, y): #compute a mask so that the prediction of the next token can only depend on the previous tokens
        batch_size, seq_len = y.shape
        mask = torch.tril(torch.ones((seq_len, seq_len), dtype = torch.bool)).expand(
            batch_size, 1, seq_len, seq_len)
        return mask & self.create_padding_mask(y)

    def create_padding_mask(self, x):
        batch_size, seq_len = x.shape #FAI UN CHECK PER VEDERE SE FA COSE SENSATE
        mask = (x != self.pad_id).unsqueeze(-2).unsqueeze(-2).expand(
                batch_size, 1, seq_len, seq_len)
        return mask

    def inference(self, x):
        #encode and then generate the output token by token greedily
        self.eval()
        with torch.no_grad():
            src_mask = self.create_padding_mask(x)
            x = self.token_embedding(x)
            x = self.positional_embedding(x)
            output_encoder = self.encoder(x)
            output = torch.ones(x.shape[0], 1).fill_(0).type_as(x) #0 is the <bos> index in the vocabulary
            for _ in range(self.max_len_answer): #CONTROLLA PERCHÉ ORA 
                trg_mask = self.create_trg_mask(output)
                output_embedding = self.token_embedding(output)
                output_embedding = self.positional_embedding(output_embedding)
                out = self.decoder(output_encoder, src_mask, output_embedding, trg_mask)
                out = self.to_logits(out) 
                argmax = 0                         #DA FARE PRESTO
                output = output.cat([output, argmax], dim = -1)
            return output

    def forward(self, x, y):
        src_mask = self.create_padding_mask(x)
        trg_mask = self.create_trg_mask(y)

        x = self.token_embedding(x)
        x = self.positional_embedding(x)
        y = self.token_embedding(y)
        y = self.positional_embedding(y)

        output_encoder = self.encoder(x, src_mask)
        return self.to_logits(self.decoder(output_encoder, src_mask, y, trg_mask))
    
    def configure_optimizers(self):# learning rate = 1x10^-4; beta1 =0.9; beta2 = 0.995 dal paper
        return torch.optim.Adam(self.parameters(), lr=1e-4, betas=(0.9, 0.995))

    def training_step(self, batch, batch_idx): #aggiungi gradient clipping nel Trainer
        x, y = batch
        y_pred = self(x, y)
        loss = F.cross_entropy(y_pred, y, ignore_index = self.pad_id)
        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self.inference(x)
        self.accuracy.update(y_pred, y)

    def validation_epoch_end(self, outputs):
        self.log('accuracy_epoch', self.accuracy.compute())
        self.accuracy.reset()


In [134]:
dm = Mathematics_DataModule(['./mathematics_dataset-v1.0/train-easy/algebra__linear_1d.txt'], batch_size = 8)

In [135]:
dm.setup()
dl = dm.train_dataloader()

In [136]:
x, y = next(iter(dl))
(x.shape, y.shape)

(torch.Size([8, 162]), torch.Size([8, 162]))

In [137]:
t1 = Transformer(256, 4, 8, 256, 0.2, 58, num_blocks_encoder = 2, num_blocks_decoder = 2)

In [138]:
t1(x, y)

query shape: torch.Size([8, 4, 162, 64])
key.T shape: torch.Size([8, 4, 64, 162])
t shape: torch.Size([8, 4, 162, 162])
mask shape: torch.Size([8, 1, 162, 162])
query shape: torch.Size([8, 4, 162, 64])
key.T shape: torch.Size([8, 4, 64, 162])
t shape: torch.Size([8, 4, 162, 162])
mask shape: torch.Size([8, 1, 162, 162])
query shape: torch.Size([8, 4, 162, 64])
key.T shape: torch.Size([8, 4, 64, 162])
t shape: torch.Size([8, 4, 162, 162])
mask shape: torch.Size([8, 1, 162, 162])
query shape: torch.Size([8, 4, 162, 64])
key.T shape: torch.Size([8, 4, 64, 162])
t shape: torch.Size([8, 4, 162, 162])
mask shape: torch.Size([8, 1, 162, 162])
query shape: torch.Size([8, 4, 162, 64])
key.T shape: torch.Size([8, 4, 64, 162])
t shape: torch.Size([8, 4, 162, 162])
mask shape: torch.Size([8, 1, 162, 162])
query shape: torch.Size([8, 4, 162, 64])
key.T shape: torch.Size([8, 4, 64, 162])
t shape: torch.Size([8, 4, 162, 162])
mask shape: torch.Size([8, 1, 162, 162])


tensor([[[-8.6860e-01, -8.8100e-01, -1.6803e-01,  ...,  4.1720e-01,
          -1.0204e+00, -8.1162e-01],
         [-3.6909e-01,  3.4496e-01,  2.8782e-01,  ...,  5.5593e-01,
           6.8619e-02, -1.0308e+00],
         [-6.7411e-01,  9.0704e-02, -5.3082e-01,  ...,  6.1977e-01,
          -9.8281e-01, -7.6199e-01],
         ...,
         [-3.7489e-01, -2.1241e-01, -1.5496e+00,  ..., -7.4588e-01,
          -8.5844e-01, -1.4641e-01],
         [-1.6210e+00, -4.2117e-02,  1.0024e-01,  ...,  2.8651e-01,
           2.9545e-01,  3.8468e-01],
         [-8.5258e-01, -1.6422e-01,  2.2411e-01,  ...,  2.1928e-01,
          -6.6220e-01, -5.0163e-01]],

        [[ 1.9412e-01,  1.2836e-02,  8.7129e-01,  ...,  7.8262e-01,
           5.0171e-02, -3.1992e-01],
         [ 3.1201e-01,  3.7068e-01,  8.1923e-02,  ..., -6.4538e-01,
          -6.0072e-01, -6.0429e-01],
         [ 6.4408e-01,  1.1403e+00,  4.1847e-02,  ...,  3.8696e-02,
          -4.4704e-02,  1.3026e-01],
         ...,
         [-7.3716e-01, -9

In [84]:
t2 = Transformer(128, 4, 8, 256, 0.2, 58, num_blocks_encoder = 2, num_blocks_decoder = 2, tp_attention = True)

In [85]:
t2(x,y)

mask after unsqueeze: torch.Size([8, 1, 1, 162])
mask after unsqueeze: torch.Size([8, 1, 1, 32])
at value shape: torch.Size([8, 4, 162, 32])
role shape: torch.Size([8, 4, 162, 32])
at value shape: torch.Size([8, 4, 162, 32])
role shape: torch.Size([8, 4, 162, 32])
at value shape: torch.Size([8, 4, 32, 32])
role shape: torch.Size([8, 4, 32, 32])
at value shape: torch.Size([8, 4, 32, 32])
role shape: torch.Size([8, 4, 32, 32])
at value shape: torch.Size([8, 4, 32, 32])
role shape: torch.Size([8, 4, 32, 32])
at value shape: torch.Size([8, 4, 32, 32])
role shape: torch.Size([8, 4, 32, 32])


tensor([[[-8.6346e-02, -9.4784e-01,  4.2047e-01,  ...,  2.6463e-01,
           8.3313e-01, -1.4773e+00],
         [ 5.1860e-01,  2.8757e-02,  2.5992e-01,  ..., -2.5352e-01,
          -3.6327e-01, -1.6211e-01],
         [ 1.3191e+00, -5.0016e-01,  1.2975e-01,  ...,  3.6880e-01,
           1.1554e-01,  6.0056e-01],
         ...,
         [ 3.0647e-01,  7.5897e-01, -2.1815e-01,  ..., -1.6927e-01,
          -1.3067e-01,  9.0310e-01],
         [ 8.8128e-01, -5.7365e-01,  9.1345e-01,  ...,  2.4984e-01,
           1.3463e+00,  5.1418e-01],
         [-7.0717e-02, -9.1214e-01,  1.3559e+00,  ...,  1.1616e-01,
          -2.7784e-01,  8.5368e-01]],

        [[-2.3930e-01, -1.4695e+00,  1.4561e-01,  ...,  1.0363e+00,
           3.3070e-01, -5.3704e-01],
         [ 2.2137e-01,  6.0087e-01, -4.7695e-02,  ..., -1.8453e+00,
           1.5709e-01, -6.2129e-02],
         [ 2.4720e-01, -9.7772e-02,  2.3021e-01,  ..., -3.1716e-01,
           3.1636e-01,  2.8233e-02],
         ...,
         [ 1.8542e+00, -5

# SOTA

# NON-SOTA